[Reference](https://medium.com/dooboolab/prisma-with-python-and-fastapi-33bf25bb20c0)

# 1. Install Prisma Client

In [1]:
!pip install prisma
!prisma generate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.6 MB/s 
     |████████████████████████████████| 84 kB 835 kB/s 
     |████████████████████████████████| 68 kB 6.7 MB/s 
     |████████████████████████████████| 80 kB 8.0 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
prisma:warn The postinstall script automatically ran `prisma generate` and did not find your `prisma/schema.prisma`.
If you have a Prisma schema file in a custom path, you will need to run
`prisma generate --schema=./path/to/your/schema.prisma` to generate Prisma Client.
If you do not have a Prisma schema file yet, you can ignore this message.



In [3]:
from prisma import Prisma
prisma = Prisma()

# 2. Install FastAPI


In [4]:
# main.py
from src.apis import apis
from src.prisma import prisma
from fastapi.middleware.gzip import GZipMiddleware
from fastapi import FastAPI
app = FastAPI()
app.add_middleware(GZipMiddleware, minimum_size=1000)
app.include_router(apis, prefix="/apis")
@app.on_event("startup")
async def startup():
    await prisma.connect()
@app.on_event("shutdown")
async def shutdown():
    await prisma.disconnect()
@app.get("/")
def read_root():
    return {"version": "1.0.0"}

In [5]:
# __init__.py
from fastapi import APIRouter
from src.apis.auth import router as authRouter
apis = APIRouter()
apis.include_router(authRouter)
__all__ = ["apis"]

In [6]:
# auth.py
import datetime
from typing import List, Optional
from fastapi import APIRouter
from prisma.models import User
from pydantic import BaseModel
from src.models.scalar import Gender
from src.prisma import prisma
from src.utils.auth import (
    encryptPassword,
    signJWT,
    validatePassword,
)
router = APIRouter()
class SignIn(BaseModel):
    email: str
    password: str
class SignInOut(BaseModel):
    token: str
    user: User
@router.post("/auth/sign-in", tags=["auth"])
async def sign_in(signIn: SignIn):
    user = await prisma.user.find_first(
        where={
            "email": signIn.email,
        }
    )
validated = validatePassword(signIn.password, user.password)
    del user.password
if validated:
        token = signJWT(user.id)
        return SignInOut(token=token, user=user)
return None
class SignUp(BaseModel):
    email: str
    password: str
    name: Optional[str] = None
    nickname: Optional[str] = None
    birthday: Optional[datetime.date] = None
    gender: Optional[Gender] = None
    phone: Optional[str] = None
@router.post("/auth/sign-up", tags=["auth"])
async def sign_up(user: SignUp):
    password = encryptPassword(user.password)
    created = await prisma.user.create(
        {
            "email": user.email,
            "password": encryptPassword(user.password),
            "name": user.name,
            "nickname": user.nickname,
            "birthDay": user.birthday,
            "gender": user.gender,
            "phone": user.phone,
        }
    )
return created